In [1404]:
import pandas as pd
import numpy as np
from time import time
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [1405]:
matplotlib.style.use('ggplot')

In [1406]:
train1 = pd.read_csv('train1.csv')
train9 = pd.read_csv('train9.csv')
test1 = pd.read_csv('test1.csv')
test9 = pd.read_csv('test9.csv')

hero_df = pd.read_csv('hero_data.csv')

In [1407]:
test1.shape

(898, 4)

In [1408]:
train1.head(2)

,user_id,hero_id,id,num_games,num_wins,kda_ratio
0,1,41,1_41,201,125,3731.707317
1,2,17,2_17,427,234,2922.058381


In [1409]:
test1.head(2)

,user_id,hero_id,id,num_games
0,3,53,3_53,278
1,4,17,4_17,180


In [1410]:
test9.head(2)

,user_id,hero_id,id,num_games,num_wins,kda_ratio
0,3,65,3_65,330,183,3418.470419
1,3,104,3_104,231,107,2522.639069


In [1411]:
hero_df.iloc[:,:12].head()

,hero_id,primary_attr,attack_type,roles,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max
0,1,agi,Melee,Carry:Escape:Nuker,200,1.50,75,0.9,-1.0,25,27,31
1,2,str,Melee,Initiator:Durable:Disabler:Jungler,200,4.25,75,0.9,-1.0,25,24,28
2,3,int,Ranged,Support:Disabler:Nuker:Durable,200,1.50,75,0.9,1.0,25,37,43
3,4,agi,Melee,Carry:Disabler:Jungler:Nuker:Initiator,200,1.50,75,0.9,0.0,25,33,39
4,5,int,Ranged,Support:Disabler:Nuker:Jungler,200,1.50,75,0.9,-1.0,25,19,25


In [1412]:
hero_df.iloc[:,12:].head()

,base_strength,base_agility,base_intelligence,strength_gain,agility_gain,intelligence_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate
0,22,22,12,1.3,2.8,1.8,150,0,1.45,310,0.5
1,25,20,18,2.8,2.2,1.6,150,900,1.70,290,0.6
2,23,23,23,2.4,2.4,2.4,400,900,1.70,310,0.6
3,23,24,18,2.7,3.0,1.7,150,900,1.70,285,0.5
4,16,16,16,2.0,1.6,2.9,600,900,1.70,275,0.5


In [1413]:
hero_df['time_base_health_regen'] = hero_df['base_health']/hero_df['base_health_regen']

In [1414]:
tmp = hero_df.groupby(['primary_attr']). \
    agg({'base_strength' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['primary_attr', 'min_base_strength', 'mean_base_strength', 'max_base_strength']
hero_df = pd.merge(hero_df, tmp, how='left', on=['primary_attr']) 

In [1415]:
tmp = hero_df.groupby(['primary_attr']). \
    agg({'base_agility' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['primary_attr', 'min_base_agility', 'mean_base_agility', 'max_base_agility']
hero_df = pd.merge(hero_df, tmp, how='left', on=['primary_attr']) 

In [1416]:
tmp = hero_df.groupby(['primary_attr']). \
    agg({'base_intelligence' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['primary_attr', 'min_base_intelligence', 'mean_base_intelligence', 'max_base_intelligence']
hero_df = pd.merge(hero_df, tmp, how='left', on=['primary_attr']) 

In [1417]:
hero_df.head(2)

,hero_id,primary_attr,attack_type,roles,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max,base_strength,base_agility,base_intelligence,strength_gain,agility_gain,intelligence_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,time_base_health_regen,min_base_strength,mean_base_strength,max_base_strength,min_base_agility,mean_base_agility,max_base_agility,min_base_intelligence,mean_base_intelligence,max_base_intelligence
0,1,agi,Melee,Carry:Escape:Nuker,200,1.50,75,0.9,-1.0,25,27,31,22,22,12,1.3,2.8,1.8,150,0,1.45,310,0.5,133.333333,14,18.648649,24,14,21.810811,30,12,16.945946,24
1,2,str,Melee,Initiator:Durable:Disabler:Jungler,200,4.25,75,0.9,-1.0,25,24,28,25,20,18,2.8,2.2,1.6,150,900,1.70,290,0.6,47.058824,17,23.162162,30,0,15.594595,23,13,17.378378,27


In [1418]:
cat_cols = ['attack_type','primary_attr']

In [1419]:
for col in cat_cols:
    dummies = pd.get_dummies(hero_df[col], prefix=col, drop_first=True)
    hero_df = pd.concat([hero_df, dummies], axis=1)

In [1420]:
roles = set()
hero_df['roles'].str.lower().str.split(':').apply(roles.update)
print (roles)

{'jungler', 'nuker', 'carry', 'escape', 'durable', 'pusher', 'support', 'disabler', 'initiator'}


In [1421]:
for role in roles:
    hero_df[role] = np.where(hero_df['roles'].str.lower().str.contains(role), 1, 0)

In [1422]:
hero_df.head()

,hero_id,primary_attr,attack_type,roles,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max,base_strength,base_agility,base_intelligence,strength_gain,agility_gain,intelligence_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,time_base_health_regen,min_base_strength,mean_base_strength,max_base_strength,min_base_agility,mean_base_agility,max_base_agility,min_base_intelligence,mean_base_intelligence,max_base_intelligence,attack_type_Ranged,primary_attr_int,primary_attr_str,jungler,nuker,carry,escape,durable,pusher,support,disabler,initiator
0,1,agi,Melee,Carry:Escape:Nuker,200,1.50,75,0.9,-1.0,25,27,31,22,22,12,1.3,2.8,1.8,150,0,1.45,310,0.5,133.333333,14,18.648649,24,14,21.810811,30,12,16.945946,24,0,0,0,0,1,1,1,0,0,0,0,0
1,2,str,Melee,Initiator:Durable:Disabler:Jungler,200,4.25,75,0.9,-1.0,25,24,28,25,20,18,2.8,2.2,1.6,150,900,1.70,290,0.6,47.058824,17,23.162162,30,0,15.594595,23,13,17.378378,27,0,0,1,1,0,0,0,1,0,0,1,1
2,3,int,Ranged,Support:Disabler:Nuker:Durable,200,1.50,75,0.9,1.0,25,37,43,23,23,23,2.4,2.4,2.4,400,900,1.70,310,0.6,133.333333,14,18.585366,25,10,16.146341,24,16,23.292683,30,1,1,0,0,1,0,0,1,0,1,1,0
3,4,agi,Melee,Carry:Disabler:Jungler:Nuker:Initiator,200,1.50,75,0.9,0.0,25,33,39,23,24,18,2.7,3.0,1.7,150,900,1.70,285,0.5,133.333333,14,18.648649,24,14,21.810811,30,12,16.945946,24,0,0,0,1,1,1,0,0,0,0,1,1
4,5,int,Ranged,Support:Disabler:Nuker:Jungler,200,1.50,75,0.9,-1.0,25,19,25,16,16,16,2.0,1.6,2.9,600,900,1.70,275,0.5,133.333333,14,18.585366,25,10,16.146341,24,16,23.292683,30,1,1,0,1,1,0,0,0,0,1,1,0


In [1423]:
#hero_df.hist(alpha=0.5, figsize=(16, 10))

In [1424]:
train = train1.append(train9, ignore_index=True)
train = train.append(test9, ignore_index=True)

In [1425]:
train9.shape

(18846, 6)

In [1426]:
test9.shape

(8082, 6)

In [1427]:
test_df.head(5)

,user_id,hero_id,id,num_games,primary_attr,attack_type,roles,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max,base_strength,base_agility,base_intelligence,strength_gain,agility_gain,intelligence_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,time_base_health_regen,min_base_strength,mean_base_strength,max_base_strength,min_base_agility,mean_base_agility,max_base_agility,min_base_intelligence,mean_base_intelligence,max_base_intelligence,attack_type_Ranged,primary_attr_int,primary_attr_str,jungler,nuker,carry,escape,durable,pusher,support,disabler,initiator,min_kda_ratio_per_game,mean_kda_ratio_per_game,max_kda_ratio_per_game,min_num_games,mean_num_games,max_num_games,min_num_wins,mean_num_wins,max_num_wins,min_win_ratio,mean_win_ratio,max_win_ratio,min_kda_ratio,mean_kda_ratio,max_kda_ratio,min_kda_ratio_pa,mean_kda_ratio_pa,max_kda_ratio_pa,min_win_ratio_pa,mean_win_ratio_pa,max_win_ratio_pa,min_num_wins_pa,mean_num_wins_pa,max_num_wins_pa,min_num_games_pa,mean_num_games_pa,max_num_games_pa,min_kda_ratio_per_game_pa,mean_kda_ratio_per_game_pa,max_kda_ratio_per_game_pa,min_kda_ratio_per_game_pa_at,mean_kda_ratio_per_game_pa_at,max_kda_ratio_per_game_pa_at,min_num_games_pa_at,mean_num_games_pa_at,max_num_games_pa_at,min_num_wins_pa_at,mean_num_wins_pa_at,max_num_wins_pa_at,min_win_ratio_pa_at,mean_win_ratio_pa_at,max_win_ratio_pa_at,min_kda_ratio_pa_at,mean_kda_ratio_pa_at,max_kda_ratio_pa_at,min_kda_ratio_h,mean_kda_ratio_pa_h,max_kda_ratio_pa_h,min_num_wins_h,mean_num_wins_h,max_num_wins_h,min_num_games_h,mean_num_games_h,max_num_games_h,min_kda_ratio_per_game_h,mean_kda_ratio_per_game_h,max_kda_ratio_per_game_h
0,3,53,3_53,0.650761,int,Ranged,Carry:Jungler:Pusher:Escape:Nuker,0.0,0.2113,0.0,0.0,0.727077,1.0,0.720114,0.779217,0.396291,0.857440,0.797603,0.430634,0.687992,0.660171,0.904029,0.877568,0.527835,0.309903,0.224340,133.333333,14,18.585366,25,10,16.146341,24,16,23.292683,30,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.368723,0.192224,0.129390,0.860955,0.732478,0.569477,0.838982,0.719986,0.544675,0.724992,0.564694,0.035893,0.786816,0.464830,0.246002,0.000000,0.517609,0.672084,0.0,0.000000,0.0,0.0,1.000000,1.0,0.0,1.000000,1.0,0.0,0.12973,0.858814,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.880578,0.422214,0.154607,0.239812,0.419917,0.466994,0.330482,0.475931,0.477120,0.453674,0.087888,0.239501
1,4,17,4_17,0.588634,int,Ranged,Carry:Escape:Nuker:Initiator:Disabler,0.0,0.2113,0.0,0.0,0.806902,1.0,0.602304,0.629054,0.396291,0.913077,0.752472,0.283793,0.592086,0.685198,0.765150,0.874764,0.527835,0.234471,0.633761,133.333333,14,18.585366,25,10,16.146341,24,16,23.292683,30,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.227363,0.122015,0.096011,0.904026,0.856522,0.711039,0.872512,0.867430,0.731492,0.726122,0.630879,0.358178,0.770975,0.476528,0.253199,0.000000,0.517609,0.672084,0.0,0.000000,0.0,0.0,1.000000,1.0,0.0,1.000000,1.0,0.0,0.12973,0.858814,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.955473,0.744350,0.375888,0.000000,0.538034,0.414569,0.000000,0.535852,0.404668,0.531172,0.207775,0.498068
2,9,51,9_51,0.454276,str,Melee,Initiator:Disabler:Durable:Nuker,0.0,0.2113,0.0,0.0,0.590616,1.0,0.693792,0.598785,0.809694,0.768511,0.374463,0.868755,0.771486,0.138159,0.042725,0.849726,0.527835,0.668615,0.224340,133.333333,17,23.162162,30,0,15.594595,23,13,17.378378,27,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.389488,0.294043,0.252812,0.675537,0.622643,0.548542,0.693006,0.630672,0.542702,0.791443,0.651078,0.172962,0.751969,0.450185,0.281424,0.892085,0.000000,0.000000,0.0,0.631755,0.0,0.0,0.590516,0.0,0.0,0.536305,0.0,1.0,0.00000,0.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.919167,0.536572,0.226918,0.000000,0.456959,0.624315,0.000000,0.469311,0.662537,0.370605,0.210665,0.436722
3,10,54,10_54,0.595601,str,Melee,Carry:Durable:Jungler:Escape:Disabler,0.0,0.2113,0.0,0.0,0.393

In [1428]:
train_df = pd.merge(train, hero_df, how='left', on=['hero_id']) 
test_df = pd.merge(test1, hero_df, how='left', on=['hero_id']) 

In [1429]:
train_df['win_ratio'] = train_df['num_wins']/train_df['num_games']

In [1430]:
train_df['kda_ratio_per_game'] = train_df['kda_ratio']/train_df['num_games']

In [1431]:
tmp = train_df.groupby(['user_id']). \
    agg({'kda_ratio_per_game' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['user_id', 'min_kda_ratio_per_game', 
               'mean_kda_ratio_per_game', 'max_kda_ratio_per_game']
train_df = pd.merge(train_df, tmp, how='left', on=['user_id']) 
test_df = pd.merge(test_df, tmp, how='left', on=['user_id']) 

In [1432]:
tmp = train_df.groupby(['user_id']). \
    agg({'num_games' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['user_id', 'min_num_games', 'mean_num_games', 'max_num_games']
train_df = pd.merge(train_df, tmp, how='left', on=['user_id']) 
test_df = pd.merge(test_df, tmp, how='left', on=['user_id']) 

In [1433]:
tmp = train_df.groupby(['user_id']). \
    agg({'num_wins' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['user_id', 'min_num_wins', 'mean_num_wins', 'max_num_wins']
train_df = pd.merge(train_df, tmp, how='left', on=['user_id']) 
test_df = pd.merge(test_df, tmp, how='left', on=['user_id']) 

In [1434]:
tmp = train_df.groupby(['user_id']). \
    agg({'win_ratio' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['user_id', 'min_win_ratio', 'mean_win_ratio', 'max_win_ratio']
train_df = pd.merge(train_df, tmp, how='left', on=['user_id']) 
test_df = pd.merge(test_df, tmp, how='left', on=['user_id']) 

In [1435]:
tmp = train_df.groupby(['user_id']). \
    agg({'kda_ratio' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['user_id', 'min_kda_ratio', 'mean_kda_ratio', 'max_kda_ratio']
train_df = pd.merge(train_df, tmp, how='left', on=['user_id']) 
test_df = pd.merge(test_df, tmp, how='left', on=['user_id']) 

In [1436]:
tmp = train_df.groupby(['primary_attr']). \
    agg({'kda_ratio' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['primary_attr', 'min_kda_ratio_pa', 'mean_kda_ratio_pa', 'max_kda_ratio_pa']
train_df = pd.merge(train_df, tmp, how='left', on=['primary_attr']) 
test_df = pd.merge(test_df, tmp, how='left', on=['primary_attr']) 

In [1437]:
tmp = train_df.groupby(['primary_attr']). \
    agg({'win_ratio' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['primary_attr', 'min_win_ratio_pa', 'mean_win_ratio_pa', 'max_win_ratio_pa']
train_df = pd.merge(train_df, tmp, how='left', on=['primary_attr']) 
test_df = pd.merge(test_df, tmp, how='left', on=['primary_attr']) 

In [1438]:
tmp = train_df.groupby(['primary_attr']). \
    agg({'num_wins' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['primary_attr', 'min_num_wins_pa', 'mean_num_wins_pa', 'max_num_wins_pa']
train_df = pd.merge(train_df, tmp, how='left', on=['primary_attr']) 
test_df = pd.merge(test_df, tmp, how='left', on=['primary_attr']) 

In [1439]:
tmp = train_df.groupby(['primary_attr']). \
    agg({'num_games' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['primary_attr', 'min_num_games_pa', 'mean_num_games_pa', 'max_num_games_pa']
train_df = pd.merge(train_df, tmp, how='left', on=['primary_attr']) 
test_df = pd.merge(test_df, tmp, how='left', on=['primary_attr']) 

In [1440]:
tmp = train_df.groupby(['primary_attr']). \
    agg({'kda_ratio_per_game' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['primary_attr', 'min_kda_ratio_per_game_pa', 
               'mean_kda_ratio_per_game_pa', 'max_kda_ratio_per_game_pa']
train_df = pd.merge(train_df, tmp, how='left', on=['primary_attr']) 
test_df = pd.merge(test_df, tmp, how='left', on=['primary_attr']) 

In [1441]:
tmp = train_df.groupby(['attack_type']). \
    agg({'kda_ratio_per_game' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['attack_type', 'min_kda_ratio_per_game_pa_at', 
               'mean_kda_ratio_per_game_pa_at', 'max_kda_ratio_per_game_pa_at']
train_df = pd.merge(train_df, tmp, how='left', on=['attack_type']) 
test_df = pd.merge(test_df, tmp, how='left', on=['attack_type']) 

In [1442]:
tmp = train_df.groupby(['attack_type']). \
    agg({'num_games' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['attack_type', 'min_num_games_pa_at', 
               'mean_num_games_pa_at', 'max_num_games_pa_at']
train_df = pd.merge(train_df, tmp, how='left', on=['attack_type']) 
test_df = pd.merge(test_df, tmp, how='left', on=['attack_type']) 

In [1443]:
tmp = train_df.groupby(['attack_type']). \
    agg({'num_wins' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['attack_type', 'min_num_wins_pa_at', 
               'mean_num_wins_pa_at', 'max_num_wins_pa_at']
train_df = pd.merge(train_df, tmp, how='left', on=['attack_type']) 
test_df = pd.merge(test_df, tmp, how='left', on=['attack_type']) 

In [1444]:
tmp = train_df.groupby(['attack_type']). \
    agg({'win_ratio' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['attack_type', 'min_win_ratio_pa_at', 
               'mean_win_ratio_pa_at', 'max_win_ratio_pa_at']
train_df = pd.merge(train_df, tmp, how='left', on=['attack_type']) 
test_df = pd.merge(test_df, tmp, how='left', on=['attack_type']) 

In [1445]:
tmp = train_df.groupby(['attack_type']). \
    agg({'kda_ratio' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['attack_type', 'min_kda_ratio_pa_at', 
               'mean_kda_ratio_pa_at', 'max_kda_ratio_pa_at']
train_df = pd.merge(train_df, tmp, how='left', on=['attack_type']) 
test_df = pd.merge(test_df, tmp, how='left', on=['attack_type']) 

In [1446]:
tmp = train_df.groupby(['hero_id']). \
    agg({'kda_ratio' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['hero_id', 'min_kda_ratio_h', 
               'mean_kda_ratio_pa_h', 'max_kda_ratio_pa_h']
train_df = pd.merge(train_df, tmp, how='left', on=['hero_id']) 
test_df = pd.merge(test_df, tmp, how='left', on=['hero_id']) 

In [1447]:
tmp = train_df.groupby(['hero_id']). \
    agg({'num_wins' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['hero_id', 'min_num_wins_h', 
               'mean_num_wins_h', 'max_num_wins_h']
train_df = pd.merge(train_df, tmp, how='left', on=['hero_id']) 
test_df = pd.merge(test_df, tmp, how='left', on=['hero_id']) 

In [1448]:
tmp = train_df.groupby(['hero_id']). \
    agg({'num_games' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['hero_id', 'min_num_games_h', 
               'mean_num_games_h', 'max_num_games_h']
train_df = pd.merge(train_df, tmp, how='left', on=['hero_id']) 
test_df = pd.merge(test_df, tmp, how='left', on=['hero_id']) 

In [1449]:
tmp = train_df.groupby(['hero_id']). \
    agg({'win_ratio' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['hero_id', 'min_win_ratio_h', 
               'mean_win_ratio_h', 'max_win_ratio_h']
train_df = pd.merge(train_df, tmp, how='left', on=['hero_id']) 
test_df = pd.merge(test_df, tmp, how='left', on=['hero_id']) 

In [1450]:
tmp = train_df.groupby(['hero_id']). \
    agg({'kda_ratio_per_game' : [np.min,np.mean,np.max]}).reset_index()
tmp.columns = ['hero_id', 'min_kda_ratio_per_game_h', 
               'mean_kda_ratio_per_game_h', 'max_kda_ratio_per_game_h']
train_df = pd.merge(train_df, tmp, how='left', on=['hero_id']) 
test_df = pd.merge(test_df, tmp, how='left', on=['hero_id']) 

In [1451]:
test_df[test_df.isnull().any(axis=1)].head(2)

,user_id,hero_id,id,num_games,primary_attr,attack_type,roles,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max,base_strength,base_agility,base_intelligence,strength_gain,agility_gain,intelligence_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,time_base_health_regen,min_base_strength,mean_base_strength,max_base_strength,min_base_agility,mean_base_agility,max_base_agility,min_base_intelligence,mean_base_intelligence,max_base_intelligence,attack_type_Ranged,primary_attr_int,primary_attr_str,jungler,nuker,carry,escape,durable,pusher,support,disabler,initiator,min_kda_ratio_per_game,mean_kda_ratio_per_game,max_kda_ratio_per_game,min_num_games,mean_num_games,max_num_games,min_num_wins,mean_num_wins,max_num_wins,min_win_ratio,mean_win_ratio,max_win_ratio,min_kda_ratio,mean_kda_ratio,max_kda_ratio,min_kda_ratio_pa,mean_kda_ratio_pa,max_kda_ratio_pa,min_win_ratio_pa,mean_win_ratio_pa,max_win_ratio_pa,min_num_wins_pa,mean_num_wins_pa,max_num_wins_pa,min_num_games_pa,mean_num_games_pa,max_num_games_pa,min_kda_ratio_per_game_pa,mean_kda_ratio_per_game_pa,max_kda_ratio_per_game_pa,min_kda_ratio_per_game_pa_at,mean_kda_ratio_per_game_pa_at,max_kda_ratio_per_game_pa_at,min_num_games_pa_at,mean_num_games_pa_at,max_num_games_pa_at,min_num_wins_pa_at,mean_num_wins_pa_at,max_num_wins_pa_at,min_win_ratio_pa_at,mean_win_ratio_pa_at,max_win_ratio_pa_at,min_kda_ratio_pa_at,mean_kda_ratio_pa_at,max_kda_ratio_pa_at,min_kda_ratio_h,mean_kda_ratio_pa_h,max_kda_ratio_pa_h,min_num_wins_h,mean_num_wins_h,max_num_wins_h,min_num_games_h,mean_num_games_h,max_num_games_h,min_win_ratio_h,mean_win_ratio_h,max_win_ratio_h,min_kda_ratio_per_game_h,mean_kda_ratio_per_game_h,max_kda_ratio_per_game_h


In [1452]:
train_df.columns.values

array(['user_id', 'hero_id', 'id', 'num_games', 'num_wins', 'kda_ratio',
       'primary_attr', 'attack_type', 'roles', 'base_health',
       'base_health_regen', 'base_mana', 'base_mana_regen', 'base_armor',
       'base_magic_resistance', 'base_attack_min', 'base_attack_max',
       'base_strength', 'base_agility', 'base_intelligence',
       'strength_gain', 'agility_gain', 'intelligence_gain',
       'attack_range', 'projectile_speed', 'attack_rate', 'move_speed',
       'turn_rate', 'time_base_health_regen', 'min_base_strength',
       'mean_base_strength', 'max_base_strength', 'min_base_agility',
       'mean_base_agility', 'max_base_agility', 'min_base_intelligence',
       'mean_base_intelligence', 'max_base_intelligence',
       'attack_type_Ranged', 'primary_attr_int', 'primary_attr_str',
       'jungler', 'nuker', 'carry', 'escape', 'durable', 'pusher',
       'support', 'disabler', 'initiator', 'win_ratio',
       'kda_ratio_per_game', 'min_kda_ratio_per_game',
       'mean

In [1453]:
skewed = ['num_games', 'base_health',
       'base_health_regen', 'base_mana', 'base_mana_regen', 'base_armor',
       'base_magic_resistance', 'base_attack_min', 'base_attack_max',
       'base_strength', 'base_agility', 'base_intelligence', 'strength_gain',
       'agility_gain', 'intelligence_gain', 'attack_range', 'projectile_speed',
       'attack_rate', 'move_speed', 'turn_rate', 'attack_type_Ranged', 'primary_attr_int',
       'primary_attr_str', 'escape', 'jungler', 'support', 'disabler', 'carry',
       'nuker', 'durable', 'pusher', 'initiator',
       'min_kda_ratio_per_game',
       'mean_kda_ratio_per_game', 'max_kda_ratio_per_game',
       'min_num_games', 'mean_num_games', 'max_num_games', 'min_num_wins',
       'mean_num_wins', 'max_num_wins', 'min_win_ratio', 'mean_win_ratio',
       'max_win_ratio', 'min_kda_ratio', 'mean_kda_ratio', 'max_kda_ratio',
       'min_kda_ratio_pa', 'mean_kda_ratio_pa', 'max_kda_ratio_pa',
       'min_win_ratio_pa', 'mean_win_ratio_pa', 'max_win_ratio_pa',
       'min_num_wins_pa', 'mean_num_wins_pa', 'max_num_wins_pa',
       'min_num_games_pa', 'mean_num_games_pa', 'max_num_games_pa',
       'min_kda_ratio_per_game_pa', 'mean_kda_ratio_per_game_pa',
       'max_kda_ratio_per_game_pa', 'min_kda_ratio_per_game_pa_at',
       'mean_kda_ratio_per_game_pa_at', 'max_kda_ratio_per_game_pa_at',
       'min_num_games_pa_at', 'mean_num_games_pa_at',
       'max_num_games_pa_at', 'min_num_wins_pa_at', 'mean_num_wins_pa_at',
       'max_num_wins_pa_at', 'min_win_ratio_pa_at', 'mean_win_ratio_pa_at',
       'max_win_ratio_pa_at', 'min_kda_ratio_pa_at',
       'mean_kda_ratio_pa_at', 'max_kda_ratio_pa_at', 'min_kda_ratio_h',
       'mean_kda_ratio_pa_h', 'max_kda_ratio_pa_h', 'min_num_wins_h',
       'mean_num_wins_h', 'max_num_wins_h', 'min_num_games_h',
       'mean_num_games_h', 'max_num_games_h', 'min_win_ratio_h',
       'mean_win_ratio_h', 'max_win_ratio_h', 'min_kda_ratio_per_game_h',
       'mean_kda_ratio_per_game_h', 'max_kda_ratio_per_game_h']

train_df = pd.DataFrame(data = train_df)
for col in skewed:
    train_df[col] = train_df[col].apply(lambda x: x if x<0 else np.log1p(x))
    test_df[col] = test_df[col].apply(lambda x: x if x<0 else np.log1p(x))

In [1454]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['num_games', 'base_health',
       'base_health_regen', 'base_mana', 'base_mana_regen', 'base_armor',
       'base_magic_resistance', 'base_attack_min', 'base_attack_max',
       'base_strength', 'base_agility', 'base_intelligence', 'strength_gain',
       'agility_gain', 'intelligence_gain', 'attack_range', 'projectile_speed',
       'attack_rate', 'move_speed', 'turn_rate', 'attack_type_Ranged', 'primary_attr_int',
       'primary_attr_str', 'escape', 'jungler', 'support', 'disabler', 'carry',
       'nuker', 'durable', 'pusher', 'initiator',
       'min_kda_ratio_per_game',
       'mean_kda_ratio_per_game', 'max_kda_ratio_per_game',
       'min_num_games', 'mean_num_games', 'max_num_games', 'min_num_wins',
       'mean_num_wins', 'max_num_wins', 'min_win_ratio', 'mean_win_ratio',
       'max_win_ratio', 'min_kda_ratio', 'mean_kda_ratio', 'max_kda_ratio',
       'min_kda_ratio_pa', 'mean_kda_ratio_pa', 'max_kda_ratio_pa',
       'min_win_ratio_pa', 'mean_win_ratio_pa', 'max_win_ratio_pa',
       'min_num_wins_pa', 'mean_num_wins_pa', 'max_num_wins_pa',
       'min_num_games_pa', 'mean_num_games_pa', 'max_num_games_pa',
       'min_kda_ratio_per_game_pa', 'mean_kda_ratio_per_game_pa',
       'max_kda_ratio_per_game_pa', 'min_kda_ratio_per_game_pa_at',
       'mean_kda_ratio_per_game_pa_at', 'max_kda_ratio_per_game_pa_at',
       'min_num_games_pa_at', 'mean_num_games_pa_at',
       'max_num_games_pa_at', 'min_num_wins_pa_at', 'mean_num_wins_pa_at',
       'max_num_wins_pa_at', 'min_win_ratio_pa_at', 'mean_win_ratio_pa_at',
       'max_win_ratio_pa_at', 'min_kda_ratio_pa_at',
       'mean_kda_ratio_pa_at', 'max_kda_ratio_pa_at', 'min_kda_ratio_h',
       'mean_kda_ratio_pa_h', 'max_kda_ratio_pa_h', 'min_num_wins_h',
       'mean_num_wins_h', 'max_num_wins_h', 'min_num_games_h',
       'mean_num_games_h', 'max_num_games_h', 'min_win_ratio_h',
       'mean_win_ratio_h', 'max_win_ratio_h', 'min_kda_ratio_per_game_h',
       'mean_kda_ratio_per_game_h', 'max_kda_ratio_per_game_h']

train_df = pd.DataFrame(data = train_df)
train_df[numerical] = scaler.fit_transform(train_df[numerical])

test_df = pd.DataFrame(data = test_df)
test_df[numerical] = scaler.fit_transform(test_df[numerical])

In [1455]:
target = train_df['kda_ratio']

In [1456]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [1457]:
def RMSE(y, pred):
    return mean_squared_error(y, pred)**0.5

In [1458]:
pd.set_option('display.max_columns', None)

In [1459]:
train_df.columns.values

array(['user_id', 'hero_id', 'id', 'num_games', 'num_wins', 'kda_ratio',
       'primary_attr', 'attack_type', 'roles', 'base_health',
       'base_health_regen', 'base_mana', 'base_mana_regen', 'base_armor',
       'base_magic_resistance', 'base_attack_min', 'base_attack_max',
       'base_strength', 'base_agility', 'base_intelligence',
       'strength_gain', 'agility_gain', 'intelligence_gain',
       'attack_range', 'projectile_speed', 'attack_rate', 'move_speed',
       'turn_rate', 'time_base_health_regen', 'min_base_strength',
       'mean_base_strength', 'max_base_strength', 'min_base_agility',
       'mean_base_agility', 'max_base_agility', 'min_base_intelligence',
       'mean_base_intelligence', 'max_base_intelligence',
       'attack_type_Ranged', 'primary_attr_int', 'primary_attr_str',
       'jungler', 'nuker', 'carry', 'escape', 'durable', 'pusher',
       'support', 'disabler', 'initiator', 'win_ratio',
       'kda_ratio_per_game', 'min_kda_ratio_per_game',
       'mean

In [1489]:
cols_to_use = ['user_id', 'hero_id', 'num_games',
       'base_health', 'base_health_regen', 'base_mana', 'base_mana_regen',
       'base_armor', 'base_magic_resistance', 'base_attack_min',
       'base_attack_max', 'base_strength', 'base_agility', 'base_intelligence',
       'strength_gain', 'agility_gain', 'intelligence_gain', 'attack_range',
       'projectile_speed', 'attack_rate', 'move_speed', 'turn_rate',
        'escape', 'initiator', 'carry', 'disabler', 'jungler', 'nuker', 'pusher',
       'support', 'durable', 
       'mean_kda_ratio_per_game',
       'mean_kda_ratio',
        'mean_kda_ratio_pa', 
        'mean_kda_ratio_per_game_pa',
       'mean_kda_ratio_per_game_pa_at',
       'mean_kda_ratio_pa_at', 
       'mean_kda_ratio_pa_h', 
       'mean_kda_ratio_per_game_h']

In [1490]:
test_df[cols_to_use].head(2)

,user_id,hero_id,num_games,base_health,base_health_regen,base_mana,base_mana_regen,base_armor,base_magic_resistance,base_attack_min,base_attack_max,base_strength,base_agility,base_intelligence,strength_gain,agility_gain,intelligence_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,escape,initiator,carry,disabler,jungler,nuker,pusher,support,durable,mean_kda_ratio_per_game,mean_kda_ratio,mean_kda_ratio_pa,mean_kda_ratio_per_game_pa,mean_kda_ratio_per_game_pa_at,mean_kda_ratio_pa_at,mean_kda_ratio_pa_h,mean_kda_ratio_per_game_h
0,3,53,0.650761,0.0,0.2113,0.0,0.0,0.727077,1.0,0.720114,0.779217,0.396291,0.857440,0.797603,0.430634,0.687992,0.660171,0.904029,0.877568,0.527835,0.309903,0.224340,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.192224,0.464830,0.517609,0.12973,0.0,1.0,0.422214,0.087888
1,4,17,0.588634,0.0,0.2113,0.0,0.0,0.806902,1.0,0.602304,0.629054,0.396291,0.913077,0.752472,0.283793,0.592086,0.685198,0.765150,0.874764,0.527835,0.234471,0.633761,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.122015,0.476528,0.517609,0.12973,0.0,1.0,0.744350,0.207775


In [1491]:
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_df[cols_to_use], 
                                                    target, 
                                                    test_size = 0.15, 
                                                    random_state = 77)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 24668 samples.
Testing set has 4354 samples.


In [1471]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer 
def kFoldCV(clf, X_train, y_train, X_test, parameters):
    n=3
    grid_obj = GridSearchCV(clf, parameters, n_jobs=-1, cv=n, verbose=10, 
                            scoring = make_scorer(mean_squared_error,greater_is_better=False))
    grid_fit = grid_obj.fit(X_train, y_train)
    best_clf = grid_fit.best_estimator_
    print('best model: ', best_clf)
    best_predictions = best_clf.predict(X_test)
    return best_clf, best_predictions

In [1472]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

params = {'n_estimators': [100, 300, 500],'max_depth': [3, 5, 11], 'learning_rate':[0.01, 0.1, 0.2], 
         'min_child_samples':[20, 40, 60, 80]}

model1 = LGBMRegressor(random_state=77, n_jobs=-1)

best_model, preds = kFoldCV(model1, X_train, y_train, X_test, params)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


KeyboardInterrupt: 

In [1085]:
print(RMSE(y_test, preds))

689.512267963


In [1473]:
def selectKImportance(model, X, k=5):
     return X.iloc[:,model.feature_importances_.argsort()[::-1][:k]]

In [1493]:
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression


model1 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=77,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
model2 = RandomForestRegressor(random_state=77, n_jobs=-1, max_depth=11, n_estimators=50, min_samples_leaf=9)
model3 = GradientBoostingRegressor(random_state=77, n_estimators=500, max_depth=5)
model4 = KNeighborsRegressor(n_jobs=-1, n_neighbors=8)

model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)
model4.fit(X_train, y_train)

preds1 = model1.predict(X_test)
preds2 = model2.predict(X_test)
preds3 = model3.predict(X_test)
preds4 = model4.predict(X_test)

print(RMSE(y_test, preds1))
print(RMSE(y_test, preds2))
print(RMSE(y_test, preds3))
print(RMSE(y_test, preds4))

563.464752082
569.475437358
575.726856164
963.296862358


In [1503]:
preds1 = model1.predict(test_df[cols_to_use])
preds2 = model2.predict(test_df[cols_to_use])
preds3 = model3.predict(test_df[cols_to_use])
preds4 = model4.predict(test_df[cols_to_use])

In [1504]:
preds = (preds1+preds2+preds3+preds4)/4

In [1505]:
sub = pd.DataFrame()
sub = pd.concat([test_df['id'], pd.Series(preds)], axis=1)
sub = sub.rename(columns={0:'kda_ratio'})
sub[['kda_ratio']] = sub[['kda_ratio']].apply(pd.to_numeric)
sub.to_csv('submission.csv', index=False)

In [1506]:
print('Done')

Done


In [244]:
X_train = train_df[cols_to_use]
X_test = test_df[cols_to_use] 

y_train = train_df['kda_ratio']

In [245]:
dtrain = xgb.DMatrix(data=X_train, label = y_train)
dtest = xgb.DMatrix(data=X_test)

In [246]:
params = {
    'eval_metric':'rmse'
}

In [247]:
# You can probably get better accuracy with rounds > 1000. 
bst = xgb.cv(params, dtrain, num_boost_round=2000, early_stopping_rounds=40,nfold=5,verbose_eval=10)

[0]	train-rmse:2625.6+4.66427	test-rmse:2634.45+25.7212
[10]	train-rmse:535.181+3.22036	test-rmse:717.351+52.089
[20]	train-rmse:486.765+4.08113	test-rmse:703.516+51.214
[30]	train-rmse:459.28+2.80468	test-rmse:698.507+52.8121
[40]	train-rmse:439.299+3.8326	test-rmse:698.755+51.2721
[50]	train-rmse:420.819+3.64496	test-rmse:699.098+52.1293
[60]	train-rmse:405.295+2.80484	test-rmse:700.454+51.2222
[70]	train-rmse:390.836+3.32859	test-rmse:700.654+51.1905


In [248]:
print(1)

1


In [213]:
bst_train = xgb.train(params, dtrain, num_boost_round=2000)

In [214]:
preds = bst_train.predict(dtest)

In [215]:
print(1)

1


In [216]:
sub = pd.DataFrame()
sub = pd.concat([test_df['id'], pd.Series(preds)], axis=1)
sub = sub.rename(columns={0:'kda_ratio'})
sub[['kda_ratio']] = sub[['kda_ratio']].apply(pd.to_numeric)
sub.to_csv('submission.csv', index=False)